In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
%matplotlib inline
#plt.ion()
num_examples = 10
run_dir = "Data"

# Generate and Save Training Data

In [ ]:
# Warning to save individual illumination data change "sys_params["run_clean"]" to False in "training_data_generation.py"
dataset_params, sys_params, facility_spec = tdg.main((_, run_dir, num_examples))

# Diagnostic

In [ ]:
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import neural_network_generation as nng
%matplotlib inline
# sys_params["run_clean"] must be false before run for diagnostic

In [ ]:
diag_dir = "Data"
iex = 0

num_examples = 1
sys_params = tdg.define_system_params(diag_dir)
sys_params["run_gen_deck"] = False
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
dataset_params["Y_train"] = Y_all[:,iex]
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

mean_power_fraction = uim.extract_run_parameters(dataset_params, facility_spec, sys_params)

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(np.squeeze(dataset_params["theta_pointings"]), np.squeeze(dataset_params["phi_pointings"]))
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_'+str(iex)+'original.png', dpi=300, bbox_inches='tight')

In [ ]:
hp.mollview(intensity_map_rms_spatial, unit="Deviation from mean (%)",flip="geo")
plt.savefig(sys_params["figure_location"]+'/deviation_from_mean_mollweide.png', dpi=300, bbox_inches='tight')
hp.graticule()

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"original.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')

In [ ]:
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
hp.mollview(intensity_map_normalized, unit="Deviation from Mean", flip="geo")

X_train = uim.imap2xtrain(intensity_map_normalized, LMAX, avg_power)
intensity_map_normalized2 = uim.xtrain2imap(X_train, LMAX, dataset_params["imap_nside"], avg_power)
hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

# Combine Datasets

In [ ]:
from netCDF4 import Dataset
from os import path
import os
import numpy as np
import netcdf_read_write as nrw
sys_params = tdg.define_system_params(run_dir)

In [ ]:
dataset1 = "Data1"
dataset2 = "Data2"

new_file = "Data3"

filename_trainingdata = dataset1 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_1 = training_data.variables["X_train"][:]
Y_1 = training_data.variables["Y_train"][:]
avg_powers_1 = training_data.variables["avg_powers"][:]
training_data.close()

filename_trainingdata = dataset2 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_2 = training_data.variables["X_train"][:]
Y_2 = training_data.variables["Y_train"][:]
avg_powers_2 = training_data.variables["avg_powers"][:]
training_data.close()

X_train = np.hstack((X_1, X_2))
Y_train = np.hstack((Y_1, Y_2))
avg_powers = np.hstack((avg_powers_1, avg_powers_2))
print(np.shape(X_train), np.shape(Y_train), np.shape(avg_powers))

filename_trainingdata = new_file + "/" + sys_params["trainingdata_filename"] 
nrw.save_training_data(X_train, Y_train, avg_powers, filename_trainingdata)

# Neural Network

In [ ]:
import neural_network_generation as nng
num_epochs = 100
num_nn = 1

In [ ]:
nn_params, nn_dataset, sys_params, nn_hyperparams = nng.main((_, run_dir, num_epochs, num_nn))

In [ ]:
fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Neural Network Random Seed')

## Compare NN

In [ ]:
import netcdf_read_write as nrw

In [ ]:
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Random Seed')

## Compare NN accuracy from weights

In [ ]:
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import neural_network_generation as nng

In [ ]:
num_nn = 1

In [ ]:
initialize_seed = np.arange(num_nn)
test_acc = np.zeros(num_nn)

sys_params = tdg.define_system_params(run_dir)
nn_params = nng.define_nn_params(num_nn)
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
#nn_params["test_fraction"] = 0.0
#nn_params["test_size"] = 0

for inn in range(num_nn):
    nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
    nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])
    nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
    y_pred = tfnn.apply_network(nn_dataset["X_test"], nn_weights)
    test_acc[inn] = np.mean(np.abs(nn_dataset["Y_test"] - y_pred.T))

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(initialize_seed, test_acc, "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Neural Network Random Seed')
plt.savefig(sys_params["figure_location"]+'/Compare_NN.png', dpi=300, bbox_inches='tight')

# Run Prediction Through Ifriit

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
%matplotlib inline
num_epochs = 10

In [ ]:
test_pred_dir = "Data_test_pred_1"

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# choose test data set
sys_params = tdg.define_system_params("Data")
nn_params = nng.define_nn_params(num_nn)
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
nn_params["test_fraction"] = 0.0
nn_params["test_size"] = 0
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)

nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])
dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
dataset_params["Y_train"] = nn_dataset["Y_train"].T
y_true = dataset_params["Y_train"]

# apply neural net to test set
sys_params = tdg.define_system_params(test_pred_dir)
y_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights)
dataset_params["Y_train"] = y_pred

print("The absolute error is: ", np.mean(np.abs(y_true - y_pred)))
print("The percentage relative error is: ", np.mean((np.abs(y_true - y_pred)) / y_true) * 100, "%")

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
iex = 0

num_examples = 1
sys_params = tdg.define_system_params(test_pred_dir)
sys_params["run_gen_deck"] = False
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
dataset_params["Y_train"] = Y_all[:,iex]
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

mean_power_fraction = uim.extract_run_parameters(dataset_params, facility_spec, sys_params)

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(np.squeeze(dataset_params["theta_pointings"]), np.squeeze(dataset_params["phi_pointings"]))
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_'+ str(iex) +'nn.png', dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+ str(iex) +"low_rms.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')

# Run PDD through NN

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
import utils_intensity_map as uim
%matplotlib inline
num_epochs = 10
num_examples = 1

In [ ]:
imap_nside = 256
LMAX = 30
input_size = 992
output_size = 16
nif_pdd_dir = "Data_pdd"

run_location = nif_pdd_dir + "/run_0"
intensity_map = nrw.read_intensity(run_location, imap_nside)
X_train, avg_powers_all = uim.create_xtrain(intensity_map, LMAX)
X_all = np.reshape(X_train, (-1,input_size))
Y_all = np.zeros(output_size)

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights("neural_network_weights" + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# param setup
nif_pdd_pred_dir = "Data_pdd_pred"
sys_params = tdg.define_system_params(nif_pdd_pred_dir)
nn_params["test_fraction"] = 0.0
nn_params["test_size"] = 0
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])

y_pred = tfnn.apply_network(nn_dataset["X_train"].T, nn_weights)

dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
dataset_params["Y_train"] = y_pred

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
iex = 0

num_examples = 1
sys_params = tdg.define_system_params(nif_pdd_pred_dir)
sys_params["run_gen_deck"] = False
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
dataset_params["Y_train"] = Y_all[:,iex]
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

mean_power_fraction = uim.extract_run_parameters(dataset_params, facility_spec, sys_params)

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(np.squeeze(dataset_params["theta_pointings"]), np.squeeze(dataset_params["phi_pointings"]))
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_nn_pdd.png', dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_nn_pdd.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')

# NN as surrogate for Ifriit

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
%matplotlib inline
num_epochs = 10
num_nn = 1
run_dir = "Data"
imap_nside = 256
LMAX = 30
filename_flipped_trainingdata = "flipped_training_data_and_labels.nc"

In [ ]:
# choose test data set
sys_params = tdg.define_system_params(run_dir)
nn_params = nng.define_nn_params(num_nn)

X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data_reversed(nn_params, sys_params, LMAX, imap_nside)
filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata
nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)
dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])

In [ ]:
sys_params = tdg.define_system_params(run_dir)
nn_params = nng.define_nn_params(num_nn)
sys_params["trainingdata_filename"] = filename_flipped_trainingdata
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
#nn_dataset = nng.normalise(nn_dataset)

In [ ]:
nn_hyperparams = nng.define_nn_hyperparams(num_epochs, num_nn)
nn_hyperparams = nng.multiple_nn(nn_params, nn_dataset, sys_params, nn_hyperparams)

## Check performance on test set

In [ ]:
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
y_pred = tfnn.apply_network(nn_dataset["X_test"], nn_weights)

In [ ]:
iex = 0

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), nn_dataset["Y_test"][iex,:] * 100.0)
#plt.plot(np.arange(LMAX), Y_all[:,iex])
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, Ifriit Inverse Ray Trace")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"original.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,iex] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate Model")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn.png", dpi=300, bbox_inches='tight')

## Find Minimum in Train set

In [ ]:
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
y_pred = tfnn.apply_network(X_all.T, nn_weights)

In [ ]:
mindex = np.argmin(np.mean(Y_all, axis=0))
print(mindex)
print(np.shape(Y_all[:,mindex]), X_all[:,mindex])

In [ ]:
fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), Y_all[:,mindex] * 100.0)
#plt.plot(np.arange(LMAX), Y_all[:,iex])
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, Ifriit Inverse Ray Trace")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(mindex)+"original.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,mindex] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate Model")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(mindex)+"nn.png", dpi=300, bbox_inches='tight')

## Search for Minima

In [ ]:
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
x_test_random = np.random.rand(1000000,16)
print(x_test_random[0,:])

In [ ]:
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
#y_pred = tfnn.apply_network(X_all.T, nn_weights)
y_pred = tfnn.apply_network(x_test_random, nn_weights)

In [ ]:
num_examples = 100
mindex = np.argmin(np.mean(y_pred, axis=0))
min_slice = np.argsort(np.mean(y_pred, axis=0))[:num_examples]
print(min_slice)
print(mindex)
print(np.shape(x_test_random[min_slice[50:60],:].T))

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,min_slice[50]] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate find minimum")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn.png", dpi=300, bbox_inches='tight')

In [ ]:
num_examples = 10
dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
#print(np.shape(dataset_params["Y_train"]))
#print(np.shape(x_test_random[min_slice,:]))
dataset_params["Y_train"] = x_test_random[min_slice[50:60],:].T

run_dir = "Data_global_min2"
sys_params = tdg.define_system_params(run_dir)

# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
iex = 0
print(min_slice[iex])

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
theta_slice = slice(0,29,4)
phi_slice   = slice(1,30,4)
power_slice = slice(3,32,4)

cone_theta_offset = dataset_params["sim_params"][theta_slice,iex]
cone_phi_offset = dataset_params["sim_params"][phi_slice,iex]
cone_powers = dataset_params["sim_params"][power_slice,iex]

beams_prev = 0
beams_tot = 0
total_power = 0
for icone in range(facility_spec['num_cones']):
    beams_per_cone = facility_spec['beams_per_cone'][icone]
    beams_tot += beams_per_cone
    total_power += cone_powers[icone] * beams_per_cone
    beams_prev += beams_per_cone
mean_power_fraction = total_power / facility_spec['nbeams']

intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(dataset_params["theta_pointings"][:,iex], dataset_params["phi_pointings"][:,iex])
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_'+str(iex)+'original.png', dpi=300, bbox_inches='tight')

print(x_test_random[min_slice[iex],:])

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,min_slice[50+iex]] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate find Minimum")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn_min_guess.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"ifriit.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')

# Check Polar Coord Generation

In [ ]:
import healpy_pointings as hpoint

In [ ]:
nr = 10000

x = np.random.rand(nr) * 2.0 - 1.0
y = np.random.rand(nr) * 2.0 - 1.0
theta_max = np.pi / 4.0

cart_x_coords = np.zeros((nr))
cart_y_coords = np.zeros((nr))
polar_p_coords = np.zeros((nr))
polar_t_coords = np.zeros((nr))
for ir in range(nr):
    cart_x_coords[ir] = x[ir]
    cart_y_coords[ir] = y[ir]

    r, offset_phi = hpoint.square2disk(x[ir], y[ir])
    offset_theta = theta_max * r

    polar_p_coords[ir] = offset_phi
    polar_t_coords[ir] = offset_theta

X = cart_x_coords
Y = cart_y_coords

fig = plt.figure()
plt.plot(X,Y,"*")

print(np.min(polar_p_coords))
print(np.max(polar_p_coords))

U = np.degrees(polar_t_coords * np.cos(polar_p_coords))
V = np.degrees(polar_t_coords * np.sin(polar_p_coords))

fig1 = plt.figure()
plt.plot(U,V,"*")

In [ ]:
nx = 50
ny = 50

x = np.linspace(-1, 1, nx)
y = np.linspace(-1, 1, ny)
theta_max = np.pi / 4.0

cart_x_coords = np.zeros((nx,ny))
cart_y_coords = np.zeros((nx,ny))
polar_p_coords = np.zeros((nx,ny))
polar_t_coords = np.zeros((nx,ny))
for ix in range(nx):
    for iy in range(ny):
        cart_x_coords[ix,iy] = x[ix]
        cart_y_coords[ix,iy] = y[iy]
        
        r, offset_phi = hpoint.square2disk(x[ix], y[iy])
        offset_theta = theta_max * r
        
        polar_p_coords[ix,iy] = offset_phi
        polar_t_coords[ix,iy] = offset_theta

X = cart_x_coords.flatten()
Y = cart_y_coords.flatten()

fig = plt.figure()
plt.plot(X,Y,"*")

phi = polar_p_coords.flatten()
theta = polar_t_coords.flatten()
U = np.degrees(theta * np.cos(phi))
V = np.degrees(theta * np.sin(phi))

fig1 = plt.figure()
plt.plot(U,V,"*")